# How to set a prior given T = 2 nonzero effects?

Assume two nonzero effects $T = 2$. We set $pve = 0.1$ so that SuSiE has certain power around $0.47$. 
We also set $pve = 0.7$ so that SuSiE has power around $0.64$.

We investigate SuSiE performance under different priors.

## Results

When $pve = 0.1$, 

* prior does not have any impact on both power and fdr. A larger prior might result in a larger cs_size.

When $pve = 0.7$,

* a larger prior might have a slightly larger power and lower fdr. Again, the cs_size will increase given a larger prior. 


In [39]:
dscout.summary[dscout.summary$pve==0.1,]

,effect_num,pve,prior,mean_corX,power,fdr,cs_size,cs_num,top_hit_rate,avg_purity
26,2,0.1,0.01,0.2152743,0.46,0.0213,1.000000,0.94,0.9787,1.0000000
92,2,0.1,0.02,0.2152743,0.47,0.0208,1.978261,0.96,0.9792,0.9901662
158,2,0.1,0.03,0.2152743,0.47,0.0208,1.673913,0.96,0.9792,0.9909194
224,2,0.1,0.05,0.2152743,0.47,0.0208,1.478261,0.96,0.9792,0.9946158
290,2,0.1,0.10,0.2152743,0.47,0.0208,1.347826,0.96,0.9792,0.9948754
356,2,0.1,0.20,0.2152743,0.47,0.0208,1.326087,0.96,0.9792,0.9948754
422,2,0.1,0.40,0.2152743,0.47,0.0208,1.304348,0.96,0.9792,0.9948754
488,2,0.1,0.50,0.2152743,0.47,0.0208,1.304348,0.96,0.9792,0.9948754
554,2,0.1,0.70,0.2152743,0.47,0.0208,1.282609,0.96,0.9792,0.9948754
620,2,0.1,0.90,0.2152743,0.47,0.0208,1.282609,0.96,0.9792,0.9948754


In [38]:
dscout.summary[dscout.summary$pve==0.7,]

,effect_num,pve,prior,mean_corX,power,fdr,cs_size,cs_num,top_hit_rate,avg_purity
50,2,0.7,0.01,0.2152743,0.63,0.0156,1.06,1.28,0.9688,0.9979029
116,2,0.7,0.02,0.2152743,0.63,0.0156,1.09,1.28,0.9688,0.9969122
182,2,0.7,0.03,0.2152743,0.63,0.0156,1.11,1.28,0.9688,0.9969122
248,2,0.7,0.05,0.2152743,0.63,0.0156,1.14,1.28,0.9688,0.9969122
314,2,0.7,0.10,0.2152743,0.63,0.0000,1.02,1.26,0.9841,0.9994104
380,2,0.7,0.20,0.2152743,0.64,0.0000,1.56,1.28,0.9688,0.9958743
446,2,0.7,0.40,0.2152743,0.64,0.0000,1.48,1.28,0.9688,0.9959935
512,2,0.7,0.50,0.2152743,0.64,0.0000,1.47,1.28,0.9688,0.9959935
578,2,0.7,0.70,0.2152743,0.64,0.0000,1.46,1.28,0.9688,0.9959935
644,2,0.7,0.90,0.2152743,0.64,0.0000,1.46,1.28,0.9688,0.9959935


## Code details

In [30]:
dscout_Q2 = readRDS('dscout_Q2.rds')
dscout_Q2 = dscout_Q2[!is.na(dscout_Q2$sim_gaussian.output.file),]
dscout_Q2 = dscout_Q2[!is.na(dscout_Q2$susie_prior.output.file),]

In [31]:
dscout_df = data.frame(dscout_Q2$sim_gaussian.effect_num, dscout_Q2$sim_gaussian.pve, dscout_Q2$susie_prior.prior,
                       dscout_Q2$score.hit, dscout_Q2$score.signal_num, dscout_Q2$score.cs_medianSize,
                       dscout_Q2$score.top_hit, dscout_Q2$sim_gaussian.mean_corX, dscout_Q2$susie_prior.avg_purity)
names(dscout_df) = c('effect_num', 'pve', 'prior','hit', 'cs_num', 'cs_size', 'top_hit', 'corX', 'avg_purity')

In [32]:
power.summary = aggregate(hit ~ effect_num + pve + prior, dscout_df, sum)
power.summary$power = power.summary$hit / (power.summary$effect_num*50)
fdr.summary = aggregate(cs_num ~ effect_num + pve + prior, dscout_df, sum)
fdr.summary$fdr = round(1 - power.summary$hit / fdr.summary$cs_num, 4)
meannonzero = function(x){mean(x[x!=0])}
setsize.summary = aggregate(cs_size ~ effect_num + pve + prior, dscout_df, meannonzero)
tophit.summary = aggregate(top_hit ~ effect_num + pve + prior, dscout_df, sum)
tophit.summary$tophit_rate = round(tophit.summary$top_hit / fdr.summary$cs_num , 4)
corX.summary = aggregate(corX ~ effect_num + pve + prior, dscout_df, mean)
cs_num.summary = aggregate(cs_num ~ effect_num + pve + prior, dscout_df, mean)
dscout_df$avg_purity[is.na(dscout_df$avg_purity)]=0
purity.summary = aggregate(avg_purity ~ effect_num + pve + prior, dscout_df, meannonzero)

In [33]:
dscout.summary = data.frame(power.summary$effect_num, power.summary$pve, power.summary$prior, corX.summary$corX,
                            power.summary$power, fdr.summary$fdr, setsize.summary$cs_size, 
                            cs_num.summary$cs_num, tophit.summary$tophit_rate, purity.summary$avg_purity)
names(dscout.summary) = c('effect_num', 'pve', 'prior', 'mean_corX','power', 
                          'fdr', 'cs_size', 'cs_num','top_hit_rate', 'avg_purity')

In [34]:
dscout.summary = dscout.summary[dscout.summary$effect_num==2, ]